In [2]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import os
from matplotlib import pyplot as plt
import sys
import time

sys.path.append('/home/juravlik/PycharmProjects/kaggle_hnm_recsys/')

from scripts.utils import create_one_hot_encoding

pd.set_option('display.max_columns', 500)

import warnings
warnings.filterwarnings("ignore")

In [3]:
pd.read_parquet('/home/juravlik/PycharmProjects/kaggle_hnm_recsys/data/features/customers/dynamic/num_days_from_purchases.parquet')

,customer_id,weeks_before_sub,customer__num_days_from_first_purchase,customer__num_days_from_last_purchase
0,7,19,600,31
1,26,19,67,8
2,38,19,589,11
3,61,19,480,74
4,96,19,576,39
...,...,...,...,...
2800008,1371975,0,538,16
2800009,1371976,0,728,35
2800010,1371977,0,733,13
2800011,1371978,0,170,92


In [48]:
df_transactions = pd.read_parquet('../../data/compressed_dataset/transactions.parquet')
df_articles = pd.read_parquet('../../data/compressed_dataset/articles.parquet')
df_customers = pd.read_parquet('../../data/compressed_dataset/customers.parquet')

article_id_int = pd.read_pickle('../../data/compressed_dataset/article_id_int.pickle')
int_article_id = pd.read_pickle('../../data/compressed_dataset/int_article_id.pickle')

customer_id_int = pd.read_pickle('../../data/compressed_dataset/customer_id_int.pickle')
int_customer_id = pd.read_pickle('../../data/compressed_dataset/int_customer_id.pickle')

In [49]:
# df_transactions = df_transactions.sort_values(['t_dat'])
# df_trans_first = df_transactions.groupby(['customer_id', 'article_id']).first().reset_index()
# df_trans_last = df_transactions.groupby(['customer_id', 'article_id']).last().reset_index()

# Date from first purchase, date from last purchase

In [50]:
df_transactions['t_dat'] = pd.to_datetime(df_transactions['t_dat'])
last_trans_date = df_transactions['t_dat'].max()


num_train_weeks = 20

for i in range(num_train_weeks-1, -1, -1):
    
    if i == 0:
        candidates_predictions = df_customers['customer_id'].unique()

    else:
        candidates_predictions = df_transactions[
            (df_transactions['t_dat'] > last_trans_date - timedelta(days=i * 7))
            & (df_transactions['t_dat'] <= last_trans_date - timedelta(days=(i-1) * 7))
        ]['customer_id'].unique()
        
    
    last_local_date = last_trans_date - timedelta(days=i * 7)
    
    train_set_st1 = df_transactions[df_transactions['t_dat'] <= last_local_date].copy()
    train_set_st1['weeks_before_sub'] = i
    
    train_set_st1 = train_set_st1[train_set_st1['customer_id'].isin(candidates_predictions)]
        
    df_trans_first_purchase = train_set_st1.groupby(['customer_id']).first().reset_index()
    df_trans_last_purchase = train_set_st1.groupby(['customer_id']).last().reset_index()
    
    train_set_st1 = train_set_st1.groupby(['customer_id']).first().reset_index()
    
    train_set_st1 = train_set_st1.merge(df_trans_first_purchase[['customer_id', 't_dat']].rename(columns={'t_dat': 'first_dat'}),
                                      on=['customer_id'])

    train_set_st1 = train_set_st1.merge(df_trans_last_purchase[['customer_id', 't_dat']].rename(columns={'t_dat': 'last_dat'}),
                                          on=['customer_id'])
    
    
    train_set_st1['customer__num_days_from_first_purchase'] = (last_local_date - train_set_st1['first_dat']).dt.days
    train_set_st1['customer__num_days_from_last_purchase'] = (last_local_date - train_set_st1['last_dat']).dt.days
    
    train_set_st1 = train_set_st1[['customer_id', 'weeks_before_sub',
                                   'customer__num_days_from_first_purchase', 'customer__num_days_from_last_purchase']]
    
    if i == num_train_weeks-1:
        all_train_set = train_set_st1
    else:
        all_train_set = pd.concat([all_train_set, train_set_st1], ignore_index=True)
    

all_train_set.to_parquet('/home/juravlik/PycharmProjects/kaggle_hnm_recsys/data/features/customers/dynamic/num_days_from_purchases.parquet',
                        index=False)

# mean_sales_channel_id

In [68]:
df_transactions['t_dat'] = pd.to_datetime(df_transactions['t_dat'])
last_trans_date = df_transactions['t_dat'].max()


num_train_weeks = 20

for i in range(num_train_weeks-1, -1, -1):
    
    if i == 0:
        candidates_predictions = df_customers['customer_id'].unique()

    else:
        candidates_predictions = df_transactions[
            (df_transactions['t_dat'] > last_trans_date - timedelta(days=i * 7))
            & (df_transactions['t_dat'] <= last_trans_date - timedelta(days=(i-1) * 7))
        ]['customer_id'].unique()
        
    
    last_local_date = last_trans_date - timedelta(days=i * 7)
    
    train_set_st1 = df_transactions[df_transactions['t_dat'] <= last_local_date].copy()
    
    train_set_st1 = train_set_st1[train_set_st1['customer_id'].isin(candidates_predictions)]
        
    train_set_st1 = train_set_st1.groupby(['customer_id'])['sales_channel_id'].mean().reset_index().rename(columns={'sales_channel_id': 'customer__mean_sales_channel_id'})
    train_set_st1['weeks_before_sub'] = i
        
    train_set_st1 = train_set_st1[['customer_id', 'weeks_before_sub',
                                   'customer__mean_sales_channel_id']]
    
    if i == num_train_weeks-1:
        all_train_set = train_set_st1
    else:
        all_train_set = pd.concat([all_train_set, train_set_st1], ignore_index=True)
    

all_train_set.to_parquet('/home/juravlik/PycharmProjects/kaggle_hnm_recsys/data/features/customers/dynamic/mean_sales_channel_id.parquet',
                        index=False)

# mean_price

In [77]:
df_transactions['t_dat'] = pd.to_datetime(df_transactions['t_dat'])
last_trans_date = df_transactions['t_dat'].max()


num_train_weeks = 20

for i in range(num_train_weeks-1, -1, -1):
    
    if i == 0:
        candidates_predictions = df_customers['customer_id'].unique()

    else:
        candidates_predictions = df_transactions[
            (df_transactions['t_dat'] > last_trans_date - timedelta(days=i * 7))
            & (df_transactions['t_dat'] <= last_trans_date - timedelta(days=(i-1) * 7))
        ]['customer_id'].unique()
        
    
    last_local_date = last_trans_date - timedelta(days=i * 7)
    
    train_set_st1 = df_transactions[df_transactions['t_dat'] <= last_local_date].copy()
    
    train_set_st1 = train_set_st1[train_set_st1['customer_id'].isin(candidates_predictions)]
        
    train_set_st1 = train_set_st1.groupby(['customer_id'])['price'].mean().reset_index().rename(columns={'price': 'customer__mean_price'})
    train_set_st1['weeks_before_sub'] = i
        
    train_set_st1 = train_set_st1[['customer_id', 'weeks_before_sub',
                                   'customer__mean_price']]
    
    if i == num_train_weeks-1:
        all_train_set = train_set_st1
    else:
        all_train_set = pd.concat([all_train_set, train_set_st1], ignore_index=True)
    

all_train_set.to_parquet('/home/juravlik/PycharmProjects/kaggle_hnm_recsys/data/features/customers/dynamic/mean_price.parquet',
                        index=False)

# num purchased items for all time / last 30d / last 7d

In [99]:
df_transactions['t_dat'] = pd.to_datetime(df_transactions['t_dat'])
last_trans_date = df_transactions['t_dat'].max()


num_train_weeks = 20

for i in range(num_train_weeks-1, -1, -1):
    
    if i == 0:
        candidates_predictions = df_customers['customer_id'].unique()

    else:
        candidates_predictions = df_transactions[
            (df_transactions['t_dat'] > last_trans_date - timedelta(days=i * 7))
            & (df_transactions['t_dat'] <= last_trans_date - timedelta(days=(i-1) * 7))
        ]['customer_id'].unique()
        
    
    last_local_date = last_trans_date - timedelta(days=i * 7)
    
    train_set_st1 = df_transactions[df_transactions['t_dat'] <= last_local_date].copy()
    
    train_set_st1 = train_set_st1[train_set_st1['customer_id'].isin(candidates_predictions)]
        
    train_set_st1_temp = train_set_st1.groupby(['customer_id'])['article_id'].count().reset_index().rename(columns={'article_id': 'customer__num_purchased_articles'})
    train_set_st1_temp = train_set_st1_temp.merge(train_set_st1.groupby(['customer_id'])['article_id'].nunique().reset_index().rename(columns={'article_id': 'customer__num_unique_purchased_articles'}),
                                        on=['customer_id'], how='left')
    
    train_set_st1_temp = train_set_st1_temp.merge(train_set_st1[train_set_st1['t_dat'] >= last_local_date-timedelta(days=90)].groupby(['customer_id'])['article_id'].count().reset_index().rename(columns={'article_id': 'customer__num_purchased_articles_last90days'}),
                                        on=['customer_id'], how='left')
    train_set_st1_temp = train_set_st1_temp.merge(train_set_st1[train_set_st1['t_dat'] >= last_local_date-timedelta(days=90)].groupby(['customer_id'])['article_id'].nunique().reset_index().rename(columns={'article_id': 'customer__num_unique_purchased_articles_last90days'}),
                                        on=['customer_id'], how='left')
    
    train_set_st1_temp = train_set_st1_temp.merge(train_set_st1[train_set_st1['t_dat'] >= last_local_date-timedelta(days=30)].groupby(['customer_id'])['article_id'].count().reset_index().rename(columns={'article_id': 'customer__num_purchased_articles_last30days'}),
                                        on=['customer_id'], how='left')
    train_set_st1_temp = train_set_st1_temp.merge(train_set_st1[train_set_st1['t_dat'] >= last_local_date-timedelta(days=30)].groupby(['customer_id'])['article_id'].nunique().reset_index().rename(columns={'article_id': 'customer__num_unique_purchased_articles_last30days'}),
                                        on=['customer_id'], how='left')
    
    
    train_set_st1_temp = train_set_st1_temp.merge(train_set_st1[train_set_st1['t_dat'] >= last_local_date-timedelta(days=7)].groupby(['customer_id'])['article_id'].count().reset_index().rename(columns={'article_id': 'customer__num_purchased_articles_last7days'}),
                                        on=['customer_id'], how='left')
    train_set_st1_temp = train_set_st1_temp.merge(train_set_st1[train_set_st1['t_dat'] >= last_local_date-timedelta(days=7)].groupby(['customer_id'])['article_id'].nunique().reset_index().rename(columns={'article_id': 'customer__num_unique_purchased_articles_last7days'}),
                                        on=['customer_id'], how='left')
    
    train_set_st1_temp['customer__unique_ratio'] = train_set_st1_temp['customer__num_unique_purchased_articles'] / train_set_st1_temp['customer__num_purchased_articles']
    train_set_st1_temp['customer__unique_ratio_last90days'] = train_set_st1_temp['customer__num_unique_purchased_articles_last90days'] / train_set_st1_temp['customer__num_purchased_articles_last90days']
    train_set_st1_temp['customer__unique_ratio_last30days'] = train_set_st1_temp['customer__num_unique_purchased_articles_last30days'] / train_set_st1_temp['customer__num_purchased_articles_last30days']
    train_set_st1_temp['customer__unique_ratio_last7days'] = train_set_st1_temp['customer__num_unique_purchased_articles_last7days'] / train_set_st1_temp['customer__num_purchased_articles_last7days']

    
    train_set_st1_temp['weeks_before_sub'] = i
        
    
        
    train_set_st1_temp = train_set_st1_temp[['customer_id', 'weeks_before_sub',
                                   'customer__num_purchased_articles', 'customer__num_unique_purchased_articles',
                                   'customer__num_purchased_articles_last90days', 'customer__num_unique_purchased_articles_last90days',
                                   'customer__num_purchased_articles_last30days', 'customer__num_unique_purchased_articles_last30days',
                                   'customer__num_purchased_articles_last7days', 'customer__num_unique_purchased_articles_last7days',
                                   'customer__unique_ratio', 'customer__unique_ratio_last30days', 'customer__unique_ratio_last7days'
                                  ]]
    
    if i == num_train_weeks-1:
        all_train_set = train_set_st1_temp
    else:
        all_train_set = pd.concat([all_train_set, train_set_st1_temp], ignore_index=True)
    
all_train_set.fillna(0, inplace=True)
all_train_set.to_parquet('/home/juravlik/PycharmProjects/kaggle_hnm_recsys/data/features/customers/dynamic/num_purchases__ratios.parquet',
                        index=False)

In [28]:
path = '/home/juravlik/PycharmProjects/kaggle_hnm_recsys/data/features/articles/dynamic/num_purchases__ratios.parquet'
df = pd.read_parquet(path)
df

,article_id,weeks_before_sub,article__num_purchased_customers,article__num_unique_purchased_customers,article__num_purchased_customers_last90days,article__num_unique_purchased_customers_last90days,article__num_purchased_customers_last30days,article__num_unique_purchased_customers_last30days,article__num_purchased_customers_last7days,article__num_unique_purchased_customers_last7days,article__num_purchased_customers_last1days,article__num_unique_purchased_customers_last1days,article__unique_ratio,article__unique_ratio_last30days,article__unique_ratio_last7days,article__unique_ratio_last1days
0,0,19,10836,6880,24,16,2,1,0,0,0,0,0.634921,0.500000,0.000000,0.000000
1,1,19,7049,5143,107,77,46,33,4,4,0,0,0.729607,0.717391,1.000000,0.000000
2,2,19,215,168,0,0,0,0,0,0,0,0,0.781395,0.000000,0.000000,0.000000
3,3,19,1028,929,7,6,1,1,1,1,0,0,0.903696,1.000000,1.000000,0.000000
4,4,19,530,451,9,9,0,0,0,0,0,0,0.850943,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1554633,105534,0,41,34,41,34,41,34,10,8,0,0,0.829268,0.829268,0.800000,0.000000
1554634,105536,0,9,9,9,9,9,9,9,9,2,2,1.000000,1.000000,1.000000,1.000000
1554635,105537,0,17,16,17,16,17,16,6,6,1,1,0.941176,0.941176,1.000000,1.000000
1554636,105538,0,35,23,35,23,35,23,31,19,13,5,0.657143,0.657143,0.612903,0.384615


In [30]:
# df['article__num_purchased_customers_last90days'] = df['article__num_purchased_customers_last90days'].astype(np.int32)
# df['article__num_unique_purchased_customers_last90days'] = df['article__num_unique_purchased_customers_last90days'].astype(np.int32)

# df['article__num_purchased_customers_last90days'] = df['article__num_purchased_customers_last90days'].astype(np.int32)
# df['article__num_unique_purchased_customers_last90days'] = df['article__num_unique_purchased_customers_last90days'].astype(np.int32)

# df['article__num_purchased_customers_last30days'] = df['article__num_purchased_customers_last30days'].astype(np.int32)
# df['article__num_unique_purchased_customers_last30days'] = df['article__num_unique_purchased_customers_last30days'].astype(np.int32)


# df['article__num_purchased_customers_last7days'] = df['article__num_purchased_customers_last7days'].astype(np.int32)
# df['article__num_unique_purchased_customers_last7days'] = df['article__num_unique_purchased_customers_last7days'].astype(np.int32)

# df['article__num_purchased_customers_last1days'] = df['article__num_purchased_customers_last1days'].astype(np.int32)
# df['article__num_unique_purchased_customers_last1days'] = df['article__num_unique_purchased_customers_last1days'].astype(np.int32)

d = dict.fromkeys(df.select_dtypes(np.int64).columns, np.int32)
df = df.astype(d)

In [31]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1554638 entries, 0 to 1554637
Data columns (total 16 columns):
 #   Column                                              Non-Null Count    Dtype  
---  ------                                              --------------    -----  
 0   article_id                                          1554638 non-null  int64  
 1   weeks_before_sub                                    1554638 non-null  int64  
 2   article__num_purchased_customers                    1554638 non-null  int64  
 3   article__num_unique_purchased_customers             1554638 non-null  int64  
 4   article__num_purchased_customers_last90days         1554638 non-null  int64  
 5   article__num_unique_purchased_customers_last90days  1554638 non-null  int32  
 6   article__num_purchased_customers_last30days         1554638 non-null  int32  
 7   article__num_unique_purchased_customers_last30days  1554638 non-null  int32  
 8   article__num_purchased_customers_last7days          

In [27]:
df.to_parquet(path, index=False)